In [1]:
import glob
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date

D:\programming\Anaconda1\envs\NN\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt
import cv2

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 16, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((16, 16, 128)))
        model.add(Conv2DTranspose(128, filter_size_g, strides=(4,4), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        '''model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())'''
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(2,2), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [ ]:
def build_discriminator():

        model = Sequential()
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.summary()

        return model

In [ ]:
def generator_containing_discriminator(g, d):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    #d_label.trainable = False
    valid = d(img)
    
    
    return Model(noise, valid)

In [ ]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [ ]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [ ]:
def get_images_classes(batch_size, data):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        rand_number = np.random.randint(0, len(data[choice_arr[i]]))
        temp_img = cv2.imread(data[choice_arr[i]][rand_number])
        
        
        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size) + class_target
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(batch_size):
        temp_img = cv2.imread(os.getcwd() + '/new256_images/abstraktnyy-ekspressionizm/303.png')
        
        X_train[i] = temp_img
        y_labels[i] = 0
    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [ ]:
def get_data():
    styles_folder = os.listdir(path=os.getcwd() + "\\new256_images\\")
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data.append(glob.glob(os.getcwd() + '\\new256_images\\' + styles_folder[i] + '\\*'))
    return data, num_styles

In [ ]:
def train_another(epochs = 100, BATCH_SIZE = 4):

    data, num_styles = get_data()
    
    generator = build_generator()
    discriminator = build_discriminator()
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    #d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    dcgan = generator_containing_discriminator(generator, discriminator)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    #d_label.trainable = True
    
    
    '''(X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))'''
    
    
    for epoch in range(epochs):
        for index in range(int(15000/BATCH_SIZE)):
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            label = index % num_styles
            
            real_images, real_labels = get_images_one_class(BATCH_SIZE, data, label)
            
            real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + label, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                #discriminator.trainable = False
                #d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss[0]))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            
            
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            #d_label.trainable = False
            
            y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 0.5, num_styles)
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, y)
            
            #d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss : %f" % (epoch, index, g_loss))
            
            if index % 50 == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, index), image)
                        image = combine_images(real_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            '''# Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()'''
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '%d.%d dis_weights.h5' % (day, month))
            #d_label.save_weights(os.getcwd() + '%d.%d dis_label_weights.h5' % (day, month))
            generator.save_weights(os.getcwd() + '%d.%d gen_weights.h5' % (day, month))

In [ ]:
img_rows = 256
img_cols = 256
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.001, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.00025, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.5)
d_optim = Adam(0.00015, beta_2 = 0.5)

In [ ]:
train_another(1000, 16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32768)             3309568   
_________________________________________________________________
batch_normalization_1 (Batch (None, 32768)             131072    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 128)       409728    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 128)       0         
__________

epoch 0 batch 59 d_loss : 1.621194
epoch 0 batch 59 g_loss : 2.401467
epoch 0 batch 60 d_loss : 1.393683
epoch 0 batch 60 g_loss : 2.513032
epoch 0 batch 61 d_loss : 1.858028
epoch 0 batch 61 g_loss : 2.829066
epoch 0 batch 62 d_loss : 1.359512
epoch 0 batch 62 g_loss : 1.875987
epoch 0 batch 63 d_loss : 1.802819
epoch 0 batch 63 g_loss : 2.396500
epoch 0 batch 64 d_loss : 1.692357
epoch 0 batch 64 g_loss : 2.172274
epoch 0 batch 65 d_loss : 1.622607
epoch 0 batch 65 g_loss : 2.339491
epoch 0 batch 66 d_loss : 1.476690
epoch 0 batch 66 g_loss : 2.407073
epoch 0 batch 67 d_loss : 1.545580
epoch 0 batch 67 g_loss : 2.540947
epoch 0 batch 68 d_loss : 1.795846
epoch 0 batch 68 g_loss : 2.808171
epoch 0 batch 69 d_loss : 1.929381
epoch 0 batch 69 g_loss : 2.583776
epoch 0 batch 70 d_loss : 1.939319
epoch 0 batch 70 g_loss : 2.524476
epoch 0 batch 71 d_loss : 1.353912
epoch 0 batch 71 g_loss : 2.383696
epoch 0 batch 72 d_loss : 1.863749
epoch 0 batch 72 g_loss : 2.195696
epoch 0 batch 73 d_l

epoch 0 batch 174 d_loss : 1.967455
epoch 0 batch 174 g_loss : 3.304684
epoch 0 batch 175 d_loss : 2.459385
epoch 0 batch 175 g_loss : 2.338881
epoch 0 batch 176 d_loss : 2.042827
epoch 0 batch 176 g_loss : 3.514590
epoch 0 batch 177 d_loss : 2.006294
epoch 0 batch 177 g_loss : 3.182179
epoch 0 batch 178 d_loss : 1.538327
epoch 0 batch 178 g_loss : 2.921204
epoch 0 batch 179 d_loss : 3.150210
epoch 0 batch 179 g_loss : 2.827681
epoch 0 batch 180 d_loss : 1.351227
epoch 0 batch 180 g_loss : 2.438637
epoch 0 batch 181 d_loss : 2.417624
epoch 0 batch 181 g_loss : 2.496074
epoch 0 batch 182 d_loss : 2.277844
epoch 0 batch 182 g_loss : 2.649145
epoch 0 batch 183 d_loss : 1.375426
epoch 0 batch 183 g_loss : 4.401439
epoch 0 batch 184 d_loss : 1.659750
epoch 0 batch 184 g_loss : 3.715039
epoch 0 batch 185 d_loss : 1.790987
epoch 0 batch 185 g_loss : 2.285489
epoch 0 batch 186 d_loss : 1.814338
epoch 0 batch 186 g_loss : 3.010561
epoch 0 batch 187 d_loss : 2.206213
epoch 0 batch 187 g_loss : 2

epoch 0 batch 288 d_loss : 14.494967
epoch 0 batch 288 g_loss : 2.456709
epoch 0 batch 289 d_loss : 1.561698
epoch 0 batch 289 g_loss : 2.525195
epoch 0 batch 290 d_loss : 14.022399
epoch 0 batch 290 g_loss : 1.812296
epoch 0 batch 291 d_loss : 1.781566
epoch 0 batch 291 g_loss : 2.816504
epoch 0 batch 292 d_loss : 14.580650
epoch 0 batch 292 g_loss : 2.004666
epoch 0 batch 293 d_loss : 1.831170
epoch 0 batch 293 g_loss : 2.373477
epoch 0 batch 294 d_loss : 14.215313
epoch 0 batch 294 g_loss : 2.956521
epoch 0 batch 295 d_loss : 1.538422
epoch 0 batch 295 g_loss : 2.512349
epoch 0 batch 296 d_loss : 14.657680
epoch 0 batch 296 g_loss : 2.429659
epoch 0 batch 297 d_loss : 1.457823
epoch 0 batch 297 g_loss : 1.955795
epoch 0 batch 298 d_loss : 14.307000
epoch 0 batch 298 g_loss : 2.368332
epoch 0 batch 299 d_loss : 1.591792
epoch 0 batch 299 g_loss : 2.082521
epoch 0 batch 300 d_loss : 14.352798
epoch 0 batch 300 g_loss : 2.460373
epoch 0 batch 301 d_loss : 1.896408
epoch 0 batch 301 g_l

epoch 0 batch 401 d_loss : 1.963258
epoch 0 batch 401 g_loss : 2.631726
epoch 0 batch 402 d_loss : 14.303303
epoch 0 batch 402 g_loss : 1.780743
epoch 0 batch 403 d_loss : 1.753612
epoch 0 batch 403 g_loss : 3.105305
epoch 0 batch 404 d_loss : 14.866653
epoch 0 batch 404 g_loss : 3.219447
epoch 0 batch 405 d_loss : 1.791931
epoch 0 batch 405 g_loss : 2.397202
epoch 0 batch 406 d_loss : 14.506153
epoch 0 batch 406 g_loss : 2.580443
epoch 0 batch 407 d_loss : 1.483978
epoch 0 batch 407 g_loss : 2.316632
epoch 0 batch 408 d_loss : 14.619417
epoch 0 batch 408 g_loss : 2.430821
epoch 0 batch 409 d_loss : 1.300352
epoch 0 batch 409 g_loss : 2.244287
epoch 0 batch 410 d_loss : 14.903282
epoch 0 batch 410 g_loss : 2.301719
epoch 0 batch 411 d_loss : 1.737413
epoch 0 batch 411 g_loss : 2.432087
epoch 0 batch 412 d_loss : 14.671814
epoch 0 batch 412 g_loss : 2.857146
epoch 0 batch 413 d_loss : 1.194594
epoch 0 batch 413 g_loss : 2.507896
epoch 0 batch 414 d_loss : 14.398377
epoch 0 batch 414 g_l

epoch 0 batch 515 d_loss : 1.804345
epoch 0 batch 515 g_loss : 2.225073
epoch 0 batch 516 d_loss : 14.363779
epoch 0 batch 516 g_loss : 2.879954
epoch 0 batch 517 d_loss : 2.230045
epoch 0 batch 517 g_loss : 2.283545
epoch 0 batch 518 d_loss : 15.329922
epoch 0 batch 518 g_loss : 1.982604
epoch 0 batch 519 d_loss : 1.634405
epoch 0 batch 519 g_loss : 2.614726
epoch 0 batch 520 d_loss : 14.565285
epoch 0 batch 520 g_loss : 1.628451
epoch 0 batch 521 d_loss : 2.090497
epoch 0 batch 521 g_loss : 2.280998
epoch 0 batch 522 d_loss : 14.088671
epoch 0 batch 522 g_loss : 2.271640
epoch 0 batch 523 d_loss : 1.679262
epoch 0 batch 523 g_loss : 1.897486
epoch 0 batch 524 d_loss : 14.151821
epoch 0 batch 524 g_loss : 2.513881
epoch 0 batch 525 d_loss : 1.735125
epoch 0 batch 525 g_loss : 2.536285
epoch 0 batch 526 d_loss : 14.488693
epoch 0 batch 526 g_loss : 1.332471
epoch 0 batch 527 d_loss : 1.971346
epoch 0 batch 527 g_loss : 1.959665
epoch 0 batch 528 d_loss : 14.874533
epoch 0 batch 528 g_l

epoch 0 batch 629 d_loss : 1.763525
epoch 0 batch 629 g_loss : 2.483496
epoch 0 batch 630 d_loss : 14.499038
epoch 0 batch 630 g_loss : 2.352737
epoch 0 batch 631 d_loss : 1.720918
epoch 0 batch 631 g_loss : 2.848587
epoch 0 batch 632 d_loss : 14.609694
epoch 0 batch 632 g_loss : 3.186772
epoch 0 batch 633 d_loss : 1.519895
epoch 0 batch 633 g_loss : 2.678503
epoch 0 batch 634 d_loss : 14.543670
epoch 0 batch 634 g_loss : 2.386643
epoch 0 batch 635 d_loss : 1.659457
epoch 0 batch 635 g_loss : 2.720764
epoch 0 batch 636 d_loss : 14.518054
epoch 0 batch 636 g_loss : 2.418659
epoch 0 batch 637 d_loss : 1.722509
epoch 0 batch 637 g_loss : 2.306873
epoch 0 batch 638 d_loss : 14.278317
epoch 0 batch 638 g_loss : 2.188461
epoch 0 batch 639 d_loss : 1.623765
epoch 0 batch 639 g_loss : 2.469453
epoch 0 batch 640 d_loss : 14.408884
epoch 0 batch 640 g_loss : 2.028205
epoch 0 batch 641 d_loss : 1.734147
epoch 0 batch 641 g_loss : 2.865947
epoch 0 batch 642 d_loss : 14.680399
epoch 0 batch 642 g_l

In [ ]:
 date = date.today()